In [ ]:
import pandas as pd
#import pylab
#import scipy
import matplotlib.pyplot as plt
import matplotlib #so I can call next line
matplotlib.style.use('ggplot')
import seaborn as sns
import numpy as np

#visuals
%matplotlib inline
pd.options.display.float_format = '{:,.2f}'.format #7,123,001.34
#'{:20,.2f}'.format #change pandas display format
pd.options.display.max_rows = 20 
#pd.get_option("display.max_rows")


from __future__ import division #so I can have float as std and int as //

In [ ]:
#in-memory grep implementation, filter lines starting wth filterKeyword
#return text string - memory hungry but fast
#http://stackoverflow.com/questions/10717504/is-it-possible-to-use-read-csv-to-read-only-specific-lines
def SimpleLineGrep(ASCIIfileName,filterKeyword):
  from cStringIO import StringIO
  s = StringIO()

  with open(ASCIIfileName) as f:
      for line in f:
          if line.startswith(filterKeyword):
              s.write(line)
  s.seek(0) # "rewind" to the beginning of the StringIO object
  # slow, test dump implementation
  # outFile = open("%s/dmp.csv" % ASCIIfileName[:ASCIIfileName.index('/')], "w")
  # outFile.write(s.getvalue())
  # outFile.close()

  return s

![](.\pics\2ndHackLogo.jpg)

# GNSS RAW ranges in Android N
## Lukasz K Bonenberg


This notebook explains how to calculate pseudo-ranges (code measurments) obtainable from the Android API 24+. This is partial implementation is based on `ProcessGnssMeas.m` Matlab code from [Google GPS Measurement Tools](https://github.com/google/gps-measurement-tools).



## Loading and reading data

Android data read by GNSSLogger output (hardware depending):


* Position, Velocity and Time (PVT) solution 
* ephemeris information
* Pseudorange/Pseudorange Rate (code)
* Accumulated Delta Range (Carrier)
* HW clock
* **raw ranges information**

For this workshop we will only focus on the last one.

## GNSS Navigation Concept

* User measures distance to four satellites
* Satellites transmit their current positions in orbit
* User solves for his position and **clock error**

<img src="https://upload.wikimedia.org/wikipedia/commons/c/c3/Bad_gdop.png", width=50%,height=60>

### Pseudorange is:

* One-way range (distance) between satellite and receiver
* Measurement of time-of-flight of coded signals
* time difference between satellite and the receiver $$L = \frac{T_{receiver \atop arrival}
-T^{satellite \atop transmission}}{c}$$.

In [ ]:
# reads GNSSLogger log file into pandas data frame
def readGNSSLogger (data_file):
  print("filtering PR from %s" % data_file)
  RawMeas = SimpleLineGrep(data_file,'Raw')
  colNames = ["Raw","ElapsedRealtimeMillis","TimeNanos","LeapSecond","TimeUncertaintyNanos","FullBiasNanos","BiasNanos","BiasUncertaintyNanos","DriftNanosPerSecond","DriftUncertaintyNanosPerSecond","HardwareClockDiscontinuityCount","SVid","TimeOffsetNanos","State","ReceivedSvTimeNanos","ReceivedSvTimeUncertaintyNanos","Cn0DbHz","PseudorangeRateMetersPerSecond","PseudorangeRateUncertaintyMetersPerSecond","AccumulatedDeltaRangeState","AccumulatedDeltaRangeMeters","AccumulatedDeltaRangeUncertaintyMeters","CarrierFrequencyHz","CarrierCycles","CarrierPhase","CarrierPhaseUncertainty","MultipathIndicator","SnrInDb","ConstellationType"]
  dataFrame = pd.read_csv(RawMeas, delimiter = ",",error_bad_lines=False,header=None,usecols=range(1,len(colNames)),
                       names= colNames,encoding = 'utf-8-sig',na_values = ["NULL",""],engine ='c')

  return dataFrame

In [ ]:
data_file = "./sampleData/S8_01.txt"
df_GNSS = readGNSSLogger(data_file)

df_GNSS.head()

## Confirming validity of the data

To obtain second level TTFF smartphone GNSSe use measurements long before TOW is decoded using AGPS.  These measurements are considered invalid in traditional GNSS understanding. To conform to the traditional approach we should only use the ranges with (1st and 3rd-bit set) indicating that [TOW is set](https://developer.android.com/reference/android/location/GnssMeasurement.html#getReceivedSvTimeNanos()). Detailed code can be checked in [GPS HAL file `gps.h`](https://github.com/android/platform_hardware_libhardware/blob/master/include/hardware/gps.h) and quick intro to [bit flags is here](http://www.icare.univ-lille1.fr/wiki/index.php/How_to_decode_bit_flags).

In [ ]:
#note that those are only basic flags 
statusFlags = {'CODE_LOCK'      : (1<<0),
'BIT_SYNC'       : (1<<1),
'SUBFRAME_SYNC'  : (1<<2),
'TOW_DECODED'    : (1<<3),
'MSEC_AMBIGUOUS' : (1<<4)}

#decode binary flags
def ShowBinaryFlags(currentGNSSstate):
    binRepresentation = '{0:14b}'.format(int(currentGNSSstate))
    return binRepresentation

#check if measurments are valid, that is CODE_LOCK and TOW_DECODED is set
def IsObservationValid(currentGNSSstate):
    #    #return (currentGNSSstate & (1 << 0))!=0 and (currentGNSSstate & (1 << 2))!=0
    return (currentGNSSstate & statusFlags['CODE_LOCK'])!=0 and (currentGNSSstate & statusFlags['TOW_DECODED'])!=0

#list all active flags
def ListActiveFlags(currentGNSSstate):

    activeFlags = list({k for k,v in statusFlags.items() 
                        if (currentGNSSstate & v !=0)})
    return activeFlags


In [ ]:
state = df_GNSS.State.iloc[0]
print """
First let's discuss how bitwise operations work. As an example let set 
'STATE_CODE_LOCK' flag by setting first bit = {0:06b}
and then lets set 'STATE_CODE_LOCK' flag using 4th bit {1:06b}. 
Great, let's now check how does it work with our data - our first recorded
state is {2} that is binary {2:012b} so following flags {3} are active.
""".format(1<<0,(1<<0) | (1<<3),state,ListActiveFlags(state))

validityOfData= map(lambda x:IsObservationValid(x),df_GNSS.State)
allGALILEOflags = map(lambda x:ShowBinaryFlags(x),df_GNSS[df_GNSS.ConstellationType==6].State)

In [ ]:
df_GAL = df_GNSS[df_GNSS.ConstellationType==6]
df_GAL = df_GAL[180:] 
#GAL got diff flags, so I will shortcut a bit
df_GPS = df_GNSS[validityOfData] #remove invalid data
df_GPS = df_GPS[df_GPS.ConstellationType==1] 

## Which satellites are we looking at?
As an example of Android N API, lets explore [Android GNSS status](https://developer.android.com/reference/android/location/GnssStatus.html).

In [ ]:
def BasicInfo(AndroidData):

  listOfSV = AndroidData.SVid.unique()
  listOfConstelations = AndroidData.ConstellationType.unique()
  GNSS_Constelations = {1:'GPS',2:'SBAS',3:'GLONASS',4:'QZSS',5:'BeiDou',6:'Galileo'}

  print 'Observing following SVs:{}\nObserved constelations: {}'.format(
      ','.join(map(str,listOfSV)),','.join([GNSS_Constelations[s] for s in listOfConstelations]))

In [ ]:
BasicInfo(df_GNSS)

df_GAL = df_GNSS[df_GNSS.ConstellationType==6]
BasicInfo(df_GAL)
BasicInfo(df_GPS)

## Calculating pseudoranges

$$L = \frac{T_{receiver \atop arrival}
-T^{satellite \atop transmission}}{c}$$.

* receiver clock $T_{receiver \atop arrival}$[ns] is calculated from `public long getTimeNanos()`
* received GNSS satelite time $T^{satellite \atop transmission}$[ns]  is calculated from `public long getReceivedSvTimeNanos()`

### In more details:

* from `GnssMeasurementEvent.Callback` we are using:
  * to caclulate receiver clock $T_{receiver \atop arrival}$[ns] GnssClock
    *  +`getTimeNanos()` - receiver clock
    * -`getFullBiasNanos(1)` referenced to GPS starting epoch (0000Z, January 6, 1980), note that we only take first epoch
    * -`getBiasNanos(1)` for sub-ns accuracy
    * -`getTimeOffsetNanos()` for hardware related delays
  * to get satelite clock reading we extract from  "Collection<GnssMeasurements>"
    * +`getReceivedSvTimeNanos()` - satellite clock, ref to GPS Week (TOW)
* all values in [ns]
* anything within 1ms is considered the same epoch

### Why use nanoseconds as a unit?

A traditional approach define GPS time as combination of GPS week and time of week. This is not possible with this approach as we obtain ranges before TOW is set. Hence our units are ns referenced to GPS starting epoch (0000Z, January 6, 1980). This creates a risk of memory/calculation problem - it is advisable to reduce this value (by subtracting current TOW value) before conducting more complex calculations.

In [ ]:
GNSS_const = {'totalWeekSecs':7*24*3600,'lightSpeed':299792458} #constants

GPSWeek = (-df_GPS.FullBiasNanos*1e-9/GNSS_const['totalWeekSecs']).astype('int')
print 'GPS week {} '.format(GPSWeek.unique())

tRx_ns  = df_GPS.TimeNanos+df_GPS.TimeOffsetNanos-df_GPS.FullBiasNanos.iloc[0]-(GPSWeek*GNSS_const['totalWeekSecs']*1e9)   
#tRx_ns  = df_GPS.TimeNanos-df_GPS.FullBiasNanos-(GPSWeek*GNSS_const['totalWeekSecs']*1e9)  +df_GPS.TimeOffsetNanos 
PR_m = (tRx_ns-df_GPS.ReceivedSvTimeNanos)*GNSS_const['lightSpeed']*1e-9
PR_m.tail()

In [ ]:
df_GPS.ReceivedSvTimeNanos.plot(title='GPS');
#df_GAL.ReceivedSvTimeNanos.plot()

In [ ]:
# sp1
plt.title = 'Compare Clocks'
plt.subplot(121)
plt.scatter(tRx_ns,df_GPS.ReceivedSvTimeNanos,);
plt.axis('equal')

# sp2
plt.subplot(122)
plt.boxplot([tRx_ns,df_GPS.ReceivedSvTimeNanos]);


### Putting measurments into epochs

Values calculated has to be presented per SV per epoch. We need to:

* calculate each epoch (anything within 1ms is considered the same epoch)
	* compute full cycle time of measurement, in milliseonds (see `ReadGnssLogger.m`)
* obtain SV range for each epoch per satellite


In [ ]:
listOfSV = df_GPS.SVid.unique()
allRxSec = (df_GPS.TimeNanos - df_GPS.FullBiasNanos)*1e-9;
epoch=allRxSec[0::len(listOfSV)]

df_PR =pd.DataFrame({'epoch': allRxSec,'SV_ID': df_GPS.SVid,'PR': PR_m})
SV_ranges = df_PR.pivot(index='epoch',columns='SV_ID', values='PR')
SV_ranges.plot(title='GPS PR');

## Putting it all together

Lets now define all of above as a single function.

In [ ]:
def CalculatePseudorange(dataFrame):

  GNSS_const = {'totalWeekSecs':7*24*3600,'lightSpeed':299792458} #constants
  GPSWeek = (-dataFrame.FullBiasNanos*1e-9/GNSS_const['totalWeekSecs']).astype('int')
  print 'GPS week {} '.format(GPSWeek.unique())

  tRx_ns  = dataFrame.TimeNanos+dataFrame.TimeOffsetNanos-dataFrame.FullBiasNanos.iloc[0]-(GPSWeek*GNSS_const['totalWeekSecs']*1e9)   
  #tRx_ns  = dataFrame.TimeNanos-dataFrame.FullBiasNanos-(GPSWeek*GNSS_const['totalWeekSecs']*1e9)  +dataFrame.TimeOffsetNanos 
  PR_m = (tRx_ns-dataFrame.ReceivedSvTimeNanos)*GNSS_const['lightSpeed']*1e-9
  PR_m.tail()

  # get epochs
  listOfSV = dataFrame.SVid.unique()
  allRxSec = (dataFrame.TimeNanos - dataFrame.FullBiasNanos)*1e-9;
  epoch=allRxSec[0::len(listOfSV)] #all epochs, how many obs in data file

  #create temp dataframe with ranges only
  df_PR =pd.DataFrame({'epoch': allRxSec,'SV_ID': dataFrame.SVid,'PR': PR_m,'ClockBias': dataFrame.BiasUncertaintyNanos})
  # split by columns
  df_PRbySV = df_PR.pivot(index='epoch',columns='SV_ID', values='PR')

  return df_PRbySV,df_PR

In [ ]:
df_GNSS = readGNSSLogger(data_file)
validityOfData= map(lambda x:IsObservationValid(x),df_GNSS.State)
df_GPS = df_GNSS[validityOfData] #remove invalid data
df_GPS = df_GPS[df_GPS.ConstellationType==1] 

GPS_PR,allData = CalculatePseudorange(df_GPS)
GPS_PR.tail()

In [ ]:
GPS_PR.plot();

## Time accuracy

* 'getBiasUncertaintyNanos()' for the clock's Bias Uncertainty (1-Sigma) in nanoseconds. Usualy 20ns if fix, or ~$2^{e-9}$ns (2s) before.
* 'getTimeUncertaintyNanos()'for the hardware clock's time Uncertainty (1-Sigma) in nanoseconds. This tends to be 0.


In [ ]:
first_SV = allData[allData.SV_ID==2]
second_SV = allData[allData.SV_ID==24]
plt.plot(first_SV.epoch,first_SV.ClockBias);

In [ ]:
plt.plot(second_SV.epoch,second_SV.ClockBias);

## Using Galileo


In [ ]:
df_GNSS = readGNSSLogger(data_file)
dfGAL = df_GNSS[df_GNSS.ConstellationType==6][380:]
BasicInfo(dfGAL)

dfGAL,allData = CalculatePseudorange(dfGAL)
dfGAL.plot();


## Problems with the accuracy of calculations

Pandas will automaticaly determine best type of variables to use. As discussed before with large numbers we might still end up with calculation errors due to rounding (float point calculation). Example below demonstrate the problem.


In [ ]:
print df_GNSS.dtypes

In [ ]:
def CheckCalculus(number):

  y=number-1
  z=number-1+1
  print 'x-y={:}\nz-x={:}'.format(x-y,x-z)

In [ ]:
x=-1151285108458178048
CheckCalculus(x)
CheckCalculus(x*1e100)

A careful approach is needed to avoid this calculation problem. Also check those additional resources:

* <http://mpmath.org/>

# Some helpful links

* [Google code](https://github.com/google/gps-measurement-tools) - this is the official code. Check here in case of any doubt. This code is in Matlab.
* [my version of GPS Measurement tools](https://github.com/DfAC/gps-measurement-tools), mostly added notes and small changes simplifying the use of script.
* [this repo](https://github.com/DfAC/AndroidGNSS)


* [useful comments on pandas](d:\tmp\Dropbox\Edu\ION_GNSS\AndroidGNSS\)
* [bitwise operations in python](https://wiki.python.org/moin/BitwiseOperators)
* [format string](https://docs.python.org/3/library/string.html)
* [format output](https://pyformat.info)
* [Rokybun blog](http://rokubun.cat/2016/06/30/android-n-preview-gnss-measurements/)
* [BlackDotGNSS blog](http://www.blackdotgnss.com/2016/09/20/ppp-with-smartphones-are-we-there-yet/)
